Favorite books on PySpark:
* Spark: The Definitive Guide https://learning.oreilly.com/library/view/spark-the-definitive/9781491912201/
* Learning Spark https://learning.oreilly.com/library/view/learning-spark-2nd/9781492050032/

Documentation:
* https://spark.apache.org/docs/latest/api/python/index.html

Data used in this notebook. TODO put a link

In [100]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pandas as pd
import os

# Creating Spark Session

In [2]:
spark = SparkSession.builder.getOrCreate()

In [3]:
spark

# Quick Overview

In [74]:
SBERBANK_BASE_DIR = "/Users/anton/datasets/sberbank_russian_housing_market"
SBERBANK_PATH_FULL = os.path.join(SBERBANK_BASE_DIR, "train.csv")
SBERBANK_PATH_SUBSET = os.path.join(SBERBANK_BASE_DIR, "train_subset.csv")

df_full = spark.read.option("inferSchema", "true").option("header", "true").csv(SBERBANK_PATH_FULL)
df_full.printSchema()

root
 |-- id: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- full_sq: integer (nullable = true)
 |-- life_sq: string (nullable = true)
 |-- floor: string (nullable = true)
 |-- max_floor: string (nullable = true)
 |-- material: string (nullable = true)
 |-- build_year: string (nullable = true)
 |-- num_room: string (nullable = true)
 |-- kitch_sq: string (nullable = true)
 |-- state: string (nullable = true)
 |-- product_type: string (nullable = true)
 |-- sub_area: string (nullable = true)
 |-- area_m: double (nullable = true)
 |-- raion_popul: integer (nullable = true)
 |-- green_zone_part: double (nullable = true)
 |-- indust_part: double (nullable = true)
 |-- children_preschool: integer (nullable = true)
 |-- preschool_quota: string (nullable = true)
 |-- preschool_education_centers_raion: integer (nullable = true)
 |-- children_school: integer (nullable = true)
 |-- school_quota: string (nullable = true)
 |-- school_education_centers_raion: integer (nu

In [75]:
with pd.option_context('display.max_columns', None): 
    display(df_full.limit(5).toPandas())

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,product_type,sub_area,area_m,raion_popul,green_zone_part,indust_part,children_preschool,preschool_quota,preschool_education_centers_raion,children_school,school_quota,school_education_centers_raion,school_education_centers_top_20_raion,hospital_beds_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,additional_education_raion,culture_objects_top_25,culture_objects_top_25_raion,shopping_centers_raion,office_raion,thermal_power_plant_raion,incineration_raion,oil_chemistry_raion,radiation_raion,railroad_terminal_raion,big_market_raion,nuclear_reactor_raion,detention_facility_raion,full_all,male_f,female_f,young_all,young_male,young_female,work_all,work_male,work_female,ekder_all,ekder_male,ekder_female,0_6_all,0_6_male,0_6_female,7_14_all,7_14_male,7_14_female,0_17_all,0_17_male,0_17_female,16_29_all,16_29_male,16_29_female,0_13_all,0_13_male,0_13_female,raion_build_count_with_material_info,build_count_block,build_count_wood,build_count_frame,build_count_brick,build_count_monolith,build_count_panel,build_count_foam,build_count_slag,build_count_mix,raion_build_count_with_builddate_info,build_count_before_1920,build_count_1921-1945,build_count_1946-1970,build_count_1971-1995,build_count_after_1995,ID_metro,metro_min_avto,metro_km_avto,metro_min_walk,metro_km_walk,kindergarten_km,school_km,park_km,green_zone_km,industrial_km,water_treatment_km,cemetery_km,incineration_km,railroad_station_walk_km,railroad_station_walk_min,ID_railroad_station_walk,railroad_station_avto_km,railroad_station_avto_min,ID_railroad_station_avto,public_transport_station_km,public_transport_station_min_walk,water_km,water_1line,mkad_km,ttk_km,sadovoe_km,bulvar_ring_km,kremlin_km,big_road1_km,ID_big_road1,big_road1_1line,big_road2_km,ID_big_road2,railroad_km,railroad_1line,zd_vokzaly_avto_km,ID_railroad_terminal,bus_terminal_avto_km,ID_bus_terminal,oil_chemistry_km,nuclear_reactor_km,radiation_km,power_transmission_line_km,thermal_power_plant_km,ts_km,big_market_km,market_shop_km,fitness_km,swim_pool_km,ice_rink_km,stadium_km,basketball_km,hospice_morgue_km,detention_facility_km,public_healthcare_km,university_km,workplaces_km,shopping_centers_km,office_km,additional_education_km,preschool_km,big_church_km,church_synagogue_km,mosque_km,theater_km,museum_km,exhibition_km,catering_km,ecology,green_part_500,prom_part_500,office_count_500,office_sqm_500,trc_count_500,trc_sqm_500,cafe_count_500,cafe_sum_500_min_price_avg,cafe_sum_500_max_price_avg,cafe_avg_price_500,cafe_count_500_na_price,cafe_count_500_price_500,cafe_count_500_price_1000,cafe_count_500_price_1500,cafe_count_500_price_2500,cafe_count_500_price_4000,cafe_count_500_price_high,big_church_count_500,church_count_500,mosque_count_500,leisure_count_500,sport_count_500,market_count_500,green_part_1000,prom_part_1000,office_count_1000,office_sqm_1000,trc_count_1000,trc_sqm_1000,cafe_count_1000,cafe_sum_1000_min_price_avg,cafe_sum_1000_max_price_avg,cafe_avg_price_1000,cafe_count_1000_na_price,cafe_count_1000_price_500,cafe_count_1000_price_1000,cafe_count_1000_price_1500,cafe_count_1000_price_2500,cafe_count_1000_price_4000,cafe_count_1000_price_high,big_church_count_1000,church_count_1000,mosque_count_1000,leisure_count_1000,sport_count_1000,market_count_1000,green_part_1500,prom_part_1500,office_count_1500,office_sqm_1500,trc_count_1500,trc_sqm_1500,cafe_count_1500,cafe_sum_1500_min_price_avg,cafe_sum_1500_max_price_avg,cafe_avg_price_1500,cafe_count_1500_na_price,cafe_count_1500_price_500,cafe_count_1500_price_1000,cafe_count_1500_price_1500,cafe_count_1500_price_2500,cafe_count_1500_price_4000,cafe_count_1500_price_high,big_church_count_1500,church_count_1500,mosque_count_1500,leisure_count_1500,sport_count_1500,market_count_1500,green_part_2000,prom_part_2000,office_count_2000,office_sqm_2000,trc_count_2000,trc_sqm_2000,cafe_count_2000,cafe_sum_2000_min_price_avg,cafe_sum_2000_max_price_av

In [76]:
df = df_full.select("timestamp",       # date of transaction
                    "full_sq",         # total area
                    "life_sq",         # living area
                    "floor",
                    "sub_area",        # district
                    "green_zone_part",
                    "indust_part",
                    "price_doc").      # sale price

In [82]:
df.agg({"price_doc": "max"}).show()

+--------------+
|max(price_doc)|
+--------------+
|     111111112|
+--------------+



In [83]:
df.agg({"price_doc": "min"}).show()

+--------------+
|min(price_doc)|
+--------------+
|        100000|
+--------------+



In [78]:
df.write.mode("overwrite").option("header", "true").csv(SBERBANK_PATH_SUBSET)

In [79]:
df = spark.read.option("inferSchema", "true").option("header", "true").csv(SBERBANK_PATH_SUBSET)
df.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- full_sq: integer (nullable = true)
 |-- life_sq: string (nullable = true)
 |-- floor: string (nullable = true)
 |-- sub_area: string (nullable = true)
 |-- green_zone_part: double (nullable = true)
 |-- indust_part: double (nullable = true)
 |-- price_doc: integer (nullable = true)



# Creating and Reading Data

In [85]:
numbers = spark.range(10)
numbers.show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
+---+



In [86]:
df = spark \
  .read \
  .option("inferSchema", "true") \
  .option("header", "true") \
  .csv(SBERBANK_PATH_SUBSET)

In [87]:
import tempfile
import os

with tempfile.NamedTemporaryFile() as f:
    f.write(b'1\t"Anna"\n2\tBoris')
    f.flush()
    os.fsync(f)

    schema = StructType([
        StructField("ID", LongType(), nullable=False),
        StructField("Name", StringType(), nullable=False)
    ])

    spark.read.format("csv").schema(schema).option('sep', '\t').load(f.name).show()

+---+-----+
| ID| Name|
+---+-----+
|  1| Anna|
|  2|Boris|
+---+-----+



In [112]:
schema = StructType([
    StructField("Name", StringType(), nullable=False),
    StructField("ID", LongType(), nullable=False)
])
row = Row("Hello", 1)
spark.createDataFrame([row], schema).show()

+-----+---+
| Name| ID|
+-----+---+
|Hello|  1|
+-----+---+



# Showing Data

In [88]:
df.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- full_sq: integer (nullable = true)
 |-- life_sq: string (nullable = true)
 |-- floor: string (nullable = true)
 |-- sub_area: string (nullable = true)
 |-- green_zone_part: double (nullable = true)
 |-- indust_part: double (nullable = true)
 |-- price_doc: integer (nullable = true)



In [95]:
df.schema

StructType(List(StructField(timestamp,TimestampType,true),StructField(full_sq,IntegerType,true),StructField(life_sq,StringType,true),StructField(floor,StringType,true),StructField(sub_area,StringType,true),StructField(green_zone_part,DoubleType,true),StructField(indust_part,DoubleType,true),StructField(price_doc,IntegerType,true)))

In [89]:
df.schema.names

['timestamp',
 'full_sq',
 'life_sq',
 'floor',
 'sub_area',
 'green_zone_part',
 'indust_part',
 'price_doc']

In [127]:
df.count() # number of rows

30471

In [90]:
df.take(2)

[Row(timestamp=datetime.datetime(2014, 3, 27, 0, 0), full_sq=35, life_sq='19', floor='3', sub_area='Ivanovskoe', green_zone_part=0.512707469, indust_part=0.000169676, price_doc=5300000),
 Row(timestamp=datetime.datetime(2014, 3, 27, 0, 0), full_sq=46, life_sq='NA', floor='12', sub_area='Tverskoe', green_zone_part=0.065444314, indust_part=7.81528e-05, price_doc=4648932)]

In [91]:
df.show(n=2)

+-------------------+-------+-------+-----+----------+---------------+-----------+---------+
|          timestamp|full_sq|life_sq|floor|  sub_area|green_zone_part|indust_part|price_doc|
+-------------------+-------+-------+-----+----------+---------------+-----------+---------+
|2014-03-27 00:00:00|     35|     19|    3|Ivanovskoe|    0.512707469| 1.69676E-4|  5300000|
|2014-03-27 00:00:00|     46|     NA|   12|  Tverskoe|    0.065444314| 7.81528E-5|  4648932|
+-------------------+-------+-------+-----+----------+---------------+-----------+---------+
only showing top 2 rows



In [92]:
df.limit(2).collect()

[Row(timestamp=datetime.datetime(2014, 3, 27, 0, 0), full_sq=35, life_sq='19', floor='3', sub_area='Ivanovskoe', green_zone_part=0.512707469, indust_part=0.000169676, price_doc=5300000),
 Row(timestamp=datetime.datetime(2014, 3, 27, 0, 0), full_sq=46, life_sq='NA', floor='12', sub_area='Tverskoe', green_zone_part=0.065444314, indust_part=7.81528e-05, price_doc=4648932)]

In [99]:
df.first()

Row(timestamp=datetime.datetime(2014, 3, 27, 0, 0), full_sq=35, life_sq='19', floor='3', sub_area='Ivanovskoe', green_zone_part=0.512707469, indust_part=0.000169676, price_doc=5300000)

In [93]:
df.limit(2).toPandas()

,timestamp,full_sq,life_sq,floor,sub_area,green_zone_part,indust_part,price_doc
0,2014-03-27,35,19,3,Ivanovskoe,0.512707,0.000170,5300000
1,2014-03-27,46,NA,12,Tverskoe,0.065444,0.000078,4648932


# Operations

> To Spark, columns are logical constructions that simply represent a value computed on a per-record basis by means of an expression. This means that to have a real value for a column, we need to have a row; and to have a row, we need to have a DataFrame. You cannot manipulate an individual column outside the context of a DataFrame

From "Spark: The Definitive Guide"

In [94]:
col('abc'), column('timestamp')

(Column<b'abc'>, Column<b'timestamp'>)

In [98]:
expr('full_sq > 35')

Column<b'(full_sq > 35)'>

In [116]:
df.select("timestamp", "price_doc").show(2)

+-------------------+---------+
|          timestamp|price_doc|
+-------------------+---------+
|2014-03-27 00:00:00|  5300000|
|2014-03-27 00:00:00|  4648932|
+-------------------+---------+
only showing top 2 rows



In [119]:
df.select(expr("green_zone_part * 100").alias("green_zone_percent"), expr("price_doc / 75 as price_usd")).show(2)

+------------------+-----------------+
|green_zone_percent|        price_usd|
+------------------+-----------------+
|        51.2707469|70666.66666666667|
| 6.544431400000001|         61985.76|
+------------------+-----------------+
only showing top 2 rows



In [121]:
df.selectExpr("*", "green_zone_part * 100 as green_zone_percent", "price_doc / 75 as price_usd").show(2)

+-------------------+-------+-------+-----+----------+---------------+-----------+---------+------------------+-----------------+
|          timestamp|full_sq|life_sq|floor|  sub_area|green_zone_part|indust_part|price_doc|green_zone_percent|        price_usd|
+-------------------+-------+-------+-----+----------+---------------+-----------+---------+------------------+-----------------+
|2014-03-27 00:00:00|     35|     19|    3|Ivanovskoe|    0.512707469| 1.69676E-4|  5300000|        51.2707469|70666.66666666667|
|2014-03-27 00:00:00|     46|     NA|   12|  Tverskoe|    0.065444314| 7.81528E-5|  4648932| 6.544431400000001|         61985.76|
+-------------------+-------+-------+-----+----------+---------------+-----------+---------+------------------+-----------------+
only showing top 2 rows



In [125]:
df.selectExpr("avg(price_doc)", "count(*)").show(2)

+----------------+--------+
|  avg(price_doc)|count(1)|
+----------------+--------+
|7123035.27773949|   30471|
+----------------+--------+



In [128]:
df.select("timestamp", lit(5).alias("Five")).show(2)

+-------------------+----+
|          timestamp|Five|
+-------------------+----+
|2014-03-27 00:00:00|   5|
|2014-03-27 00:00:00|   5|
+-------------------+----+
only showing top 2 rows



In [132]:
df.withColumn("more_green_than_indust", expr("green_zone_part > indust_part")).show(2)

+-------------------+-------+-------+-----+----------+---------------+-----------+---------+----------------------+
|          timestamp|full_sq|life_sq|floor|  sub_area|green_zone_part|indust_part|price_doc|more_green_than_indust|
+-------------------+-------+-------+-----+----------+---------------+-----------+---------+----------------------+
|2014-03-27 00:00:00|     35|     19|    3|Ivanovskoe|    0.512707469| 1.69676E-4|  5300000|                  true|
|2014-03-27 00:00:00|     46|     NA|   12|  Tverskoe|    0.065444314| 7.81528E-5|  4648932|                  true|
+-------------------+-------+-------+-----+----------+---------------+-----------+---------+----------------------+
only showing top 2 rows



In [133]:
df.withColumnRenamed("timestamp", "selling_date").columns

['selling_date',
 'full_sq',
 'life_sq',
 'floor',
 'sub_area',
 'green_zone_part',
 'indust_part',
 'price_doc']

In [134]:
df.drop("sub_area").columns

['timestamp',
 'full_sq',
 'life_sq',
 'floor',
 'green_zone_part',
 'indust_part',
 'price_doc']

In [135]:
df.withColumn("green_zone_percent", expr("green_zone_part * 100").cast("long")).show(2)

+-------------------+-------+-------+-----+----------+---------------+-----------+---------+------------------+
|          timestamp|full_sq|life_sq|floor|  sub_area|green_zone_part|indust_part|price_doc|green_zone_percent|
+-------------------+-------+-------+-----+----------+---------------+-----------+---------+------------------+
|2014-03-27 00:00:00|     35|     19|    3|Ivanovskoe|    0.512707469| 1.69676E-4|  5300000|                51|
|2014-03-27 00:00:00|     46|     NA|   12|  Tverskoe|    0.065444314| 7.81528E-5|  4648932|                 6|
+-------------------+-------+-------+-----+----------+---------------+-----------+---------+------------------+
only showing top 2 rows



In [136]:
df.filter(col("floor") > 5).count()

17030

In [142]:
df.where(col("floor") > 5).where(expr("full_sq > 100")).select(expr("avg(price_doc)").cast("int")).show()

+---------------------------+
|CAST(avg(price_doc) AS INT)|
+---------------------------+
|                   21172852|
+---------------------------+



In [143]:
df.select("sub_area").distinct().count()

146

In [145]:
df.sample(withReplacement=False, fraction=0.0002, seed=42).show()

+-------------------+-------+-------+-----+--------------------+---------------+-----------+---------+
|          timestamp|full_sq|life_sq|floor|            sub_area|green_zone_part|indust_part|price_doc|
+-------------------+-------+-------+-----+--------------------+---------------+-----------+---------+
|2012-10-19 00:00:00|     53|     29|   11|             Mar'ino|    0.188712765|0.090799103|  7500000|
|2014-09-19 00:00:00|     64|     NA|    9|Poselenie Moskovskij|    0.492839845|0.075778755|  6004650|
|2013-09-10 00:00:00|     39|     18|   12|Pokrovskoe Stresh...|    0.183968555|0.150817539|  6650000|
|2012-03-21 00:00:00|     40|     18|   16|            Brateevo|    0.398794394|        0.0|  6600000|
+-------------------+-------+-------+-----+--------------------+---------------+-----------+---------+



In [148]:
data_frames = df.randomSplit([0.7, 0.2, 0.1], seed=42)
len(data_frames), data_frames[0].count(), data_frames[1].count(), data_frames[2].count()

(3, 21367, 6116, 2988)

In [149]:
data_frames[0].union(data_frames[1]).union(data_frames[2]).count()

30471

In [150]:
df.sort(col("price_doc").desc()).show(5)

+-------------------+-------+-------+-----+--------------------+---------------+-----------+---------+
|          timestamp|full_sq|life_sq|floor|            sub_area|green_zone_part|indust_part|price_doc|
+-------------------+-------+-------+-----+--------------------+---------------+-----------+---------+
|2012-05-10 00:00:00|     55|     31|   12|         Teplyj Stan|    0.426396297|0.002211665|111111112|
|2015-03-12 00:00:00|    220|    144|    3|         Presnenskoe|    0.068202173|0.042031587| 95122496|
|2013-04-19 00:00:00|    185|     66|   33|             Ramenki|    0.169625419|0.019508796| 91066096|
|2014-05-29 00:00:00|    184|     85|   10|             Ramenki|    0.169625419|0.019508796| 80777440|
|2013-03-06 00:00:00|    206|    128|    2|Pokrovskoe Stresh...|    0.183968555|0.150817539| 78802248|
+-------------------+-------+-------+-----+--------------------+---------------+-----------+---------+
only showing top 5 rows



# Debugging

In [21]:
df.sort('timestamp').explain()

== Physical Plan ==
Sort [timestamp#11 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(timestamp#11 ASC NULLS FIRST, 200)
   +- FileScan csv [id#10,timestamp#11,full_sq#12,life_sq#13,floor#14,max_floor#15,material#16,build_year#17,num_room#18,kitch_sq#19,state#20,product_type#21,sub_area#22,area_m#23,raion_popul#24,green_zone_part#25,indust_part#26,children_preschool#27,preschool_quota#28,preschool_education_centers_raion#29,children_school#30,school_quota#31,school_education_centers_raion#32,school_education_centers_top_20_raion#33,... 268 more fields] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/Users/anton/datasets/sberbank_russian_housing_market/train.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<id:int,timestamp:timestamp,full_sq:int,life_sq:string,floor:string,max_floor:string,materi...
